In [ ]:
%%capture
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" #se non va, prova %pip install

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

EOS_TOKEN = "<|endoftext|>"

def formatting_prompts_func(examples):
    instructions = examples["Context"] # note : these are my columns
    inputs = examples["Question"] # this one
    outputs = examples["SQLite3_Query"] # and this one
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = sql_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts,}

max_seq_length = 2048 # we support rope scaling internally
dtype = None
load_in_4bit = True # 4 bit quantization to reduce memory


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto",
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

sql_prompt = """Below is input question that user ask, context is given to help user's question, generate SQLite3 response for user's question.

### Input:
{}

### Context:
{}

### SQLite3 Response:
{}"""

df = pd.read_excel('/content/drive/MyDrive/file_compl.xlxs')
dataset = Dataset.from_pandas(df)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
validation_dataset = train_test_split['test']
dataset = dataset.map(formatting_prompts_func, batched=True)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(  
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60, 
        learning_rate = 2e-4,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

train_result = trainer.train()
model.generate("fine_tuned_model")

# trainer.model.save_pretrained("fine_tuned_model")
# tokenizer.save_pretrained("fine_tuned_model")

print("done")